In [ ]:
import pandas as pd
train=pd.read_csv('/kaggle/input/predict-cltv-of-a-customer/train_BRCpofr.csv')
train.head()

In [ ]:
test=pd.read_csv('/kaggle/input/predict-cltv-of-a-customer/test_koRSKBP.csv')
test.head()

In [ ]:
train.nunique()

In [ ]:
train.info()

In [ ]:
train=train.drop(['id'],axis=1)
train.head()

In [ ]:
test=test.drop(['id'],axis=1)
test.head()

In [ ]:
train['marital_status']=train['marital_status'].replace({0:'Single',1:'Married'})
train.head()

In [ ]:
test['marital_status']=test['marital_status'].replace({0:'Single',1:'Married'})
test.head()

In [ ]:
train['gender'].value_counts()

In [ ]:
test['gender'].value_counts()

In [ ]:
train['area'].value_counts()

In [ ]:
test['area'].value_counts()

In [ ]:
train['qualification'].value_counts()

In [ ]:
test['qualification'].value_counts()

In [ ]:
train['marital_status'].value_counts()

In [ ]:
test['marital_status'].value_counts()

In [ ]:
train['num_policies'].value_counts()

In [ ]:
test['num_policies'].value_counts()

In [ ]:
train['num_policies']=train['num_policies'].replace({'1':0,'More than 1':1})
train.head()

In [ ]:
test['num_policies']=test['num_policies'].replace({'1':0,'More than 1':1})
test.head()

In [ ]:
train['policy'].value_counts()

In [ ]:
test['policy'].value_counts()

In [ ]:
train['type_of_policy'].value_counts()

In [ ]:
test['type_of_policy'].value_counts()

In [ ]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [ ]:
import numpy as np
train, NAlist = reduce_mem_usage(train)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

In [ ]:
test, NAlist = reduce_mem_usage(test)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

In [ ]:
!pip install flaml
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
y = train.pop('cltv')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,shuffle=True)

In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
automl.fit(X_train, y_train, task="regression",metric='rmse',time_budget=3600*3)

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best rmse on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
from flaml.ml import sklearn_metric_loss_score
print('train rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_train), y_train))
print(' test rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_test), y_test))

In [ ]:
y_pred = automl.predict(test)
y_pred[:5]

In [ ]:
df = pd.DataFrame(y_pred,columns=['cltv'])
df.head()

In [ ]:
sol=pd.read_csv('/kaggle/input/predict-cltv-of-a-customer/sample_submission_9zqft7i/sample_submission.csv')
sol.head()

In [ ]:
sol['cltv']=df['cltv']
sol.head()

In [ ]:
sol.to_csv('./rmse.csv',index=False)